<h2> <center> Tutorial on <br> The Hydrogen atom  </center> </h2>

<br>
<h3> <center> Reinhard J. Maurer </center> </h3>

<div class="col-md-12">
                    <h2> The Hydrogen atom </h2>
                    <div class="col-md-12">
                        <div class="col-md-4">
                            <div class="col-md-10">
                                <img src="hydrogen.jpeg" alt="Motivation">
                            </div>
                        </div>
                        <div class="col-md-8">
                            <p>  Schrödinger equation (SE): $\quad\hat{H}\psi = E\psi $</p>
                            <p class="">  
                            SE: Hamilton ('energy') operator $\hat{H}$ defines the energy and wave function $\psi$  </p>
                            <p class=""> $$ \hat{H} = -\frac{\hbar^2}{2m_e}\nabla^2-\frac{Ze^2}{4\pi\epsilon_0 r}  $$</p>
                            <p> </p>
                            <div class="col-md-12">                            
                            <div class="col-md-12">
                                <b> energies ('eigenvalues') </b>
                                <p> $$ E_{nl} = -\frac{Z^2 m_e e^4}{32\pi^2 \epsilon_0^2 \hbar^2}\frac{1}{n^2}  \quad\text{where}\quad n = 1,2, \dots $$  </p>
                            </div>                        
                            <div class="col-md-12">
                                <b> wave functions ('eigenfunctions')</b> 
                                <p> $$ \psi_{nlm}(r,\theta,\phi) = R_{nl}(r)Y^m_l(\theta,\phi)  $$  </p>
                                <p> $$ R_{nl}(r) = -\Big( \big(\frac{2Z}{na}\big)^3 \frac{(n-l-1)!}{2n[(n+1)!]^3} \Big) \rho^l L^{2l+1}_{n+l}(\rho)\cdot e^{-\rho/2}  $$ </p>
                                <p> $$ \quad\text{with}\quad \rho=(2Z/na)r \quad\text{and}\quad a=4\pi\epsilon_0\hbar^2/m_e e^2  $$</p>
                                <p>   </p>
                            </div>
                            </div>                                
                        </div>
                    </div>
</div>                    


In [ ]:
from scipy.special import sph_harm 
from scipy.special import assoc_laguerre
from scipy.special import factorial
import numpy as np

def S(l,m,theta,phi):
    if m>0:
        prefac = ((-1)**m)/sqrt(2.0)
        sph = sph_harm(m,l,theta,phi) + np.conjugate(sph_harm(m,l,theta,phi))
        return prefac*sph        
    elif m<0:
        prefac = ((-1)**m)/(1.j*sqrt(2.0))
        m = abs(m)
        sph = sph_harm(m,l,theta,phi) - np.conjugate(sph_harm(m,l,theta,phi))
        return prefac*sph
    else:
        return sph_harm(0,l,theta,phi)
    
#L_n^k
def L(n,k,x): 
    return assoc_laguerre(x,n,k)

def R(n,l,Z, r):
    prefac = ((2.*Z)**3)*( factorial(n-l-1)/(2*n*factorial(n+l)**3) )
    rho = (2.*Z/n) * r
    return prefac * (rho**l) * L(n+l,2*l+1,rho) * np.exp(-rho/2)
    
    

In [ ]:
def calc_n_states(n_max):
    n_states = 0
    for n in range(n_max+1):
        for l in range(0,n):
            for m in range(-l,l+1):
                n_states += 1
    return n_states


def nlm_2_index(n_ref,l_ref,m_ref, n_max=1000):
    i = 0
    for n in range(1,n_max):
        for l in range(0,n):
            for m in range(-l,l+1):
                if n==n_ref and l==l_ref and m==m_ref:
                    index = i
                    return index
                i += 1
    

def index_2_nlm(index, n_max=1000):
    i = 0
    for n in range(n_max+1):
        for l in range(0,n):
            for m in range(-l,l+1):
                if i==index:
                    n_ref = n
                    l_ref = l
                    m_ref = m
                    return (n_ref, l_ref, m_ref)
                i += 1


def do_calcs(Z, r_max=20.0, n_r_points=100,  n_states=100):
    
    n_theta_points = 30
    n_phi_points= 10
    
    r_grid = np.logspace(0,r_max,n_r_points)
    theta_grid = np.linspace(0,0.2*np.pi,n_theta_points)
    phi_grid = np.linspace(0,np.pi,n_phi_points)

    Ry = 13.60569 #eV
    n_max = n_states      
        
    E = np.zeros(n_states)
    #psi = np.zeros([n_states, n_r_points, n_theta_points, n_phi_points])
    psi_r = np.zeros([n_states, n_r_points])
    for i in range(n_states):
        n, l, m = index_2_nlm(i, n_max)
        E[i] = -Z / n**2
        for ri, r in enumerate(r_grid):
            psi_r[i,ri] = R(n,l,Z,r)
            #for ti,theta in enumerate(theta_grid):
                #for pi, phi in enumerate(phi_grid):
                    #psi[i,ri,ti,pi] = R(n,l,Z,r) * S(l,m,theta,phi)
    
    dens_r = psi_r*psi_r

    prob_r = psi_r*psi_r*r*r
    #dens = np.conjugate(psi)*psi
    
    return E, psi_r, dens_r, prob_r

def pot(Z,r):
    return -Z/r

In [ ]:
n_max = 2
l_max = n_max-1
Z = 1

r_max=2.0
n_r_points=2000
x = np.linspace(0, r_max, n_r_points)
x2 = np.linspace(0, r_max*100, n_r_points)
p = pot(Z, x2)
results = do_calcs(Z=1, n_r_points=n_r_points, n_states=10*10)
E = results[0]
print(E)

In [ ]:
%matplotlib inline 
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
    
def plot_hydrogen(x, pot, E, psi_r, dens_r, prob_r, n=1, l=0, m=0):
        """
        plots the energies, the wave function and the probability density with interactive handles.
        """

        import matplotlib.gridspec as gridspec
        plt.figure(figsize=(11, 8), dpi= 80, facecolor='w', edgecolor='k') # figsize determines the actual size of the figure
        gs1 = gridspec.GridSpec(3, 3)
        gs1.update(left=0.05, right=0.90, wspace=0.35,hspace=0.25)
        ax1 = plt.subplot(gs1[: ,0])
        ax2 = plt.subplot(gs1[0, 1:])
        ax3 = plt.subplot(gs1[1, 1:])
        ax4 = plt.subplot(gs1[2, 1:])        

        #plot potential and energy levels
        x2 = x*10
        ax1.plot(x2, pot, color='gray')
        ax1.plot(np.zeros(len(E)), E, lw=0.0, marker='_',ms=4000, color='blue')
        #ax1.plot(0, E[n], lw=0.0, marker='_',ms=4000, color='red')
        ax1.set_ylim(E[-1]-0.1,0.01)
        ax1.axhline(0.0,color='black')

        index = nlm_2_index(n,l,m)
        print(index)
        ax2.plot(x, psi_r[index,:])
        ax3.plot(x, dens_r[index,:])
        ax4.plot(x, prob_r[index,:])

        #figure out boundaries
        start=0; stop=0
        for i, p in enumerate(pot):
            if (p>-0.001 and p<0.001):
                start=x[i-1]
                break
        for i, p in enumerate(pot):
            if (p>-0.001 and p<0.001):
                stop=x[i+1]


        #Labeling of x and y axes
        ax1.set_xlabel('x',fontsize=14)
        ax1.set_ylabel('Energy [eV]',fontsize=14)
        ax2.set_ylabel(r'wave function $\psi(r)$',fontsize=14)
        ax3.set_ylabel(r'density $|\psi|^2$(r)',fontsize=14)
        ax4.set_ylabel(r'Rad. Probability $|\psi|^2\cdot r^2$(r)',fontsize=14)

        ax2.set_xlabel(r'x',fontsize=14)
        ax3.set_xlabel(r'x',fontsize=14)

        #Show the final result
        plt.show()
        return plot_hydrogen


In [ ]:
#plot_hydrogen(data,n=1, l=0, m=0) 

E, psi_r, dens_r, prob_r = results

#plot_hydrogen( x, p, E, psi_r, dens_r, prob_r, n=2, l=1, m=0)   


print("Remember that l<n and m=-l to +l, otherwise the visualization fails")
interact(plot_hydrogen, x=fixed(x), pot=fixed(p), E=fixed(E), psi_r=fixed(psi_r), dens_r=fixed(dens_r), 
                prob_r=fixed(prob_r), n=range(1,6), l=range(0,5), m=range(-5,5))
#interact(plot_hydrogen, (fixed(x), fixed(p), fixed(E), fixed(psi_r), fixed(dens_r), fixed(prob_r), 1, 0, 0))

#interact(plot_hydrogen, data, n=range(1,n_max), l=range(0,n_max-1), m=range(-l_max,l_max) )   

